In [2]:
import pandas as pd

Import all the files

In [3]:
visits = pd.read_csv('visits.csv',
                     parse_dates=[1])
cart = pd.read_csv('cart.csv',
                   parse_dates=[1])

checkout = pd.read_csv('checkout.csv',
                       parse_dates=[1])
purchase = pd.read_csv('purchase.csv',
                       parse_dates=[1])

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Step 1: Inspect the DataFrames using `print` and `head`

In [11]:
visits.info()
cart.info()
checkout.info()
purchase.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   user_id     2000 non-null   object        
 1   visit_time  2000 non-null   datetime64[ns]
dtypes: datetime64[ns](1), object(1)
memory usage: 31.4+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 348 entries, 0 to 347
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   user_id    348 non-null    object        
 1   cart_time  348 non-null    datetime64[ns]
dtypes: datetime64[ns](1), object(1)
memory usage: 5.6+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226 entries, 0 to 225
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   user_id        226 non-null    object        
 1   checkout_time  226 non-null  

In [5]:
print(visits.head(5))
print(cart.head(5))
print(checkout.head(5))
print(purchase.head(5))

                                user_id          visit_time
0  943647ef-3682-4750-a2e1-918ba6f16188 2017-04-07 15:14:00
1  0c3a3dd0-fb64-4eac-bf84-ba069ce409f2 2017-01-26 14:24:00
2  6e0b2d60-4027-4d9a-babd-0e7d40859fb1 2017-08-20 08:23:00
3  6879527e-c5a6-4d14-b2da-50b85212b0ab 2017-11-04 18:15:00
4  a84327ff-5daa-4ba1-b789-d5b4caf81e96 2017-02-27 11:25:00
                                user_id           cart_time
0  2be90e7c-9cca-44e0-bcc5-124b945ff168 2017-11-07 20:45:00
1  4397f73f-1da3-4ab3-91af-762792e25973 2017-05-27 01:35:00
2  a9db3d4b-0a0a-4398-a55a-ebb2c7adf663 2017-03-04 10:38:00
3  b594862a-36c5-47d5-b818-6e9512b939b3 2017-09-27 08:22:00
4  a68a16e2-94f0-4ce8-8ce3-784af0bbb974 2017-07-26 15:48:00
                                user_id       checkout_time
0  d33bdc47-4afa-45bc-b4e4-dbe948e34c0d 2017-06-25 09:29:00
1  4ac186f0-9954-4fea-8a27-c081e428e34e 2017-04-07 20:11:00
2  3c9c78a7-124a-4b77-8d2e-e1926e011e7d 2017-07-13 11:38:00
3  89fe330a-8966-4756-8f7c-3bdbcd47279a 

Step 2: Left merging visits and cart

In [6]:
visits_cart = pd.merge(visits, cart, how='left')
print(visits_cart.head())

                                user_id          visit_time  \
0  943647ef-3682-4750-a2e1-918ba6f16188 2017-04-07 15:14:00   
1  0c3a3dd0-fb64-4eac-bf84-ba069ce409f2 2017-01-26 14:24:00   
2  6e0b2d60-4027-4d9a-babd-0e7d40859fb1 2017-08-20 08:23:00   
3  6879527e-c5a6-4d14-b2da-50b85212b0ab 2017-11-04 18:15:00   
4  a84327ff-5daa-4ba1-b789-d5b4caf81e96 2017-02-27 11:25:00   

            cart_time  
0                 NaT  
1 2017-01-26 14:44:00  
2 2017-08-20 08:31:00  
3                 NaT  
4                 NaT  


Step 3: How long is `visits_cart`?

In [7]:
visits_cart.shape

(2000, 3)

Step 4: How many timestamps are null for `cart_time`?

In [8]:
visits_cart.cart_time.isnull().sum()

1652

Step 5: What percentage only visited?

In [12]:
visits_cart.cart_time.isnull().sum() / visits_cart.shape[0] * 100

82.6

Step 6: What percentage placed a t-shirt in their cart but did not checkout?

In [17]:
checkout.info()
(visits_cart.shape[0] - visits_cart.cart_time.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226 entries, 0 to 225
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   user_id        226 non-null    object        
 1   checkout_time  226 non-null    datetime64[ns]
dtypes: datetime64[ns](1), object(1)
memory usage: 3.7+ KB


348

In [19]:
(visits_cart.shape[0] - visits_cart.cart_time.isnull().sum() - checkout.shape[0]) / visits_cart.shape[0] * 100

6.1

Step 7: Merge it all together

In [21]:
funnel = visits_cart.merge(cart, how='left').merge(checkout, how='left').merge(purchase, how='left')
funnel.head(10)

,user_id,visit_time,cart_time,checkout_time,purchase_time
0,943647ef-3682-4750-a2e1-918ba6f16188,2017-04-07 15:14:00,NaT,NaT,NaT
1,0c3a3dd0-fb64-4eac-bf84-ba069ce409f2,2017-01-26 14:24:00,2017-01-26 14:44:00,2017-01-26 14:54:00,2017-01-26 15:08:00
2,6e0b2d60-4027-4d9a-babd-0e7d40859fb1,2017-08-20 08:23:00,2017-08-20 08:31:00,NaT,NaT
3,6879527e-c5a6-4d14-b2da-50b85212b0ab,2017-11-04 18:15:00,NaT,NaT,NaT
4,a84327ff-5daa-4ba1-b789-d5b4caf81e96,2017-02-27 11:25:00,NaT,NaT,NaT
5,7bc62113-d51d-4e6b-85e0-1cf29e3de74a,2017-06-03 20:05:00,NaT,NaT,NaT
6,6f22cd44-bc63-4449-a9af-2651859da2f9,2017-03-18 23:09:00,NaT,NaT,NaT
7,49a6399b-c095-4e42-92eb-af90abe2bb6a,2017-01-10 12:09:00,NaT,NaT,NaT
8,653c095d-fd74-40dd-ae3b-5a69a2613629,2017-06-27 19:21:00,NaT,NaT,NaT
9,3d9fdac1-aae3-4a01-b448-934d12047468,2017-09-24 06:43:00,NaT,NaT,NaT


Step 8: % of users who got to checkout but did not purchase

In [22]:
(funnel.shape[0] - funnel.checkout_time.isnull().sum()) / funnel.shape[0] * 100

15.844402277039848

Step 9: check each part of the funnel, let's print all 3 of them again

In [23]:
print(visits.head())
print(cart.head())
print(checkout.head())

                                user_id          visit_time
0  943647ef-3682-4750-a2e1-918ba6f16188 2017-04-07 15:14:00
1  0c3a3dd0-fb64-4eac-bf84-ba069ce409f2 2017-01-26 14:24:00
2  6e0b2d60-4027-4d9a-babd-0e7d40859fb1 2017-08-20 08:23:00
3  6879527e-c5a6-4d14-b2da-50b85212b0ab 2017-11-04 18:15:00
4  a84327ff-5daa-4ba1-b789-d5b4caf81e96 2017-02-27 11:25:00
                                user_id           cart_time
0  2be90e7c-9cca-44e0-bcc5-124b945ff168 2017-11-07 20:45:00
1  4397f73f-1da3-4ab3-91af-762792e25973 2017-05-27 01:35:00
2  a9db3d4b-0a0a-4398-a55a-ebb2c7adf663 2017-03-04 10:38:00
3  b594862a-36c5-47d5-b818-6e9512b939b3 2017-09-27 08:22:00
4  a68a16e2-94f0-4ce8-8ce3-784af0bbb974 2017-07-26 15:48:00
                                user_id       checkout_time
0  d33bdc47-4afa-45bc-b4e4-dbe948e34c0d 2017-06-25 09:29:00
1  4ac186f0-9954-4fea-8a27-c081e428e34e 2017-04-07 20:11:00
2  3c9c78a7-124a-4b77-8d2e-e1926e011e7d 2017-07-13 11:38:00
3  89fe330a-8966-4756-8f7c-3bdbcd47279a 

*The weakest part of the funnel is clearly getting a person who visited the site to add a tshirt to their cart. Once they've added a t-shirt to their cart it is fairly likely they end up purchasing it. A suggestion could be to make the add-to-cart button more prominent on the front page.*


Step 10: adding new column

In [26]:
funnel['time_to_purchase'] = funnel.purchase_time - funnel.visit_time


Step 11: examine the results

In [27]:
funnel.head(10)

,user_id,visit_time,cart_time,checkout_time,purchase_time,time_to_purchase
0,943647ef-3682-4750-a2e1-918ba6f16188,2017-04-07 15:14:00,NaT,NaT,NaT,NaT
1,0c3a3dd0-fb64-4eac-bf84-ba069ce409f2,2017-01-26 14:24:00,2017-01-26 14:44:00,2017-01-26 14:54:00,2017-01-26 15:08:00,0 days 00:44:00
2,6e0b2d60-4027-4d9a-babd-0e7d40859fb1,2017-08-20 08:23:00,2017-08-20 08:31:00,NaT,NaT,NaT
3,6879527e-c5a6-4d14-b2da-50b85212b0ab,2017-11-04 18:15:00,NaT,NaT,NaT,NaT
4,a84327ff-5daa-4ba1-b789-d5b4caf81e96,2017-02-27 11:25:00,NaT,NaT,NaT,NaT
5,7bc62113-d51d-4e6b-85e0-1cf29e3de74a,2017-06-03 20:05:00,NaT,NaT,NaT,NaT
6,6f22cd44-bc63-4449-a9af-2651859da2f9,2017-03-18 23:09:00,NaT,NaT,NaT,NaT
7,49a6399b-c095-4e42-92eb-af90abe2bb6a,2017-01-10 12:09:00,NaT,NaT,NaT,NaT
8,653c095d-fd74-40dd-ae3b-5a69a2613629,2017-06-27 19:21:00,NaT,NaT,NaT,NaT
9,3d9fdac1-aae3-4a01-b448-934d12047468,2017-09-24 06:43:00,NaT,NaT,NaT,NaT


Step 12: average time to purchase

In [28]:
funnel.time_to_purchase.mean()

Timedelta('0 days 00:43:12.380952380')